In [1]:
import xarray as xr

ml_wind = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2",
    # chunks={"time": 48},
    # decode_times=False,
    consolidated=True,
)

ml_wind = ml_wind[["u_component_of_wind", "v_component_of_wind", "temperature", "vertical_velocity"]]

print(f"Model wind dataset size {ml_wind.nbytes / (1024 ** 4):.2f} TiB")
ml_wind

Model wind dataset size 308.72 TiB


<xarray.Dataset>
Dimensions:              (time: 552264, level: 37, latitude: 721,
                          longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 1959-01-01 ... 2021-12-31T23:0...
Data variables:
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [13]:
wind_slice = ml_wind.convert_calendar("noleap")
wind_slice = wind_slice.sel(level=slice(150, 1000), time=slice("2012-01-01T00:00", "2022", 24 * 365))
wind_slice = wind_slice.mean("time")

print(f"Wind slice size {wind_slice.nbytes / (1024 ** 2):.02f} MiB")
wind_slice

Wind slice size 0.39 GiB


<xarray.Dataset>
Dimensions:              (level: 25, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 150 175 200 225 250 ... 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
Data variables:
    u_component_of_wind  (level, latitude, longitude) float32 dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (level, latitude, longitude) float32 dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    temperature          (level, latitude, longitude) float32 dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    vertical_velocity    (level, latitude, longitude) float32 dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>

In [14]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    wind_slice.to_netcdf("dataset.nc")

[########################################] | 100% Completed | 165.31 s
